# Importando bibliotecas e dataset 

In [1]:
import re
import emoji
import string
import pandas as pd
import numpy as np
import spacy
import nltk
import pt_core_news_lg
from textblob import TextBlob
from spellchecker import SpellChecker
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
from torch.utils.data import DataLoader, SequentialSampler
from transformers import TFBertForSequenceClassification, BertTokenizer
import torch

d:\ciencia_de_dados\projetos\tera_projeto\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/erika_hilton.csv', on_bad_lines='skip', sep=';', encoding='utf-8')

C:\Users\55199\AppData\Local\Temp\ipykernel_18580\1077869811.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/erika_hilton.csv', on_bad_lines='skip', sep=';', encoding='utf-8')


In [3]:
df.shape

(22505, 49)

# Filtrar tweets?
Pensei em filtrar os tweets, deixando somente os tweets direcionados a Erika, e para remover duplicatas.
Nessa fase não apliquei somente pra termos mais tweets para análise

In [3]:
df_usuarios = df[df['user'] != "https://twitter.com/ErikakHilton"]

In [4]:
df_usuarios.shape

(19390, 49)

# 1.Limpeza

In [4]:
# Removendo "não" das stopwords
STOP_WORDS.remove('não')

# Instanciando spacy/pt_core_news_lg - corpus completo
nlp = spacy.load('pt_core_news_lg')

In [8]:
print(STOP_WORDS)

{'doze', 'fazeis', 'e', 'depois', 'antes', 'com', 'pode', 'último', 'três', 'lá', 'menor', 'tipo', 'porquanto', 'todo', 'final', 'parece', 'ora', 'vem', 'faz', 'segundo', 'ser', 'outras', 'isso', 'fora', 'disso', 'estivestes', 'pôde', 'terceiro', 'vezes', 'aquelas', 'sexta', 'em', 'portanto', 'tens', 'forma', 'geral', 'contudo', 'vos', 'pouca', 'sabe', 'treze', 'apoia', 'à', 'quarta', 'dentro', 'poderá', 'comprida', 'usa', 'vais', 'no', 'seis', 'fazes', 'aí', 'local', 'vão', 'cujo', 'primeiro', 'comprido', 'devem', 'estes', 'muito', 'boa', 'nove', 'quem', 'outros', 'muitos', 'vós', 'sem', 'quinta', 'pegar', 'apenas', 'zero', 'pelo', 'além', 'agora', 'talvez', 'acerca', 'daquela', 'teu', 'caminho', 'porquê', 'cada', 'quatro', 'seus', 'tanta', 'quanto', 'após', 'ir', 'adeus', 'grande', 'mais', 'dez', 'te', 'esses', 'num', 'novos', 'tudo', 'suas', 'perto', 'dezasseis', 'numa', 'onze', 'elas', 'quero', 'tivemos', 'veja', 'estiveram', 'ou', 'nosso', 'partir', 'logo', 'os', 'possivelmente', 

In [5]:
tweets = df_usuarios['rawContent']

In [7]:
tweets.sample(10)

11744                       @ErikakHilton @silviokiwi DIVA
9395     @ErikakHilton É mais fácil o eleitor mudar de ...
12798                @ErikakHilton https://t.co/Qd5cKP4qm2
18809    @ErikakHilton @cortezpsol @GuilhermeBoulos @bf...
20870    @ErikakHilton Tô indo mami. Agora é a hora \n1...
6699     @ErikakHilton É bom lembrar que se gastar mais...
17650    @ErikakHilton @_danielaabade Ler os comentário...
13622              @ErikakHilton é de muita HIPOCRISIA !!!
3602                 @ErikakHilton #ForaBolsonaroMentiroso
6051     @ErikakHilton PQP!!!!\nPQP!!!!\nPQP!!!!\nPQP!!...
Name: rawContent, dtype: object

In [6]:
# Função para remover URL
def remover_url(texto):
    texto = re.sub(r'http\S+', '', texto)
    return texto

In [7]:
# Função para remover contas
def remover_twiters(texto):
    texto = re.sub(r'@\w+', '@MENÇÃO', texto)
    return texto

In [8]:
# Função para remover pontuações
def remover_pontuacao(texto):
    texto = re.sub(f"[{re.escape(string.punctuation)}]", "", texto)
    return texto

In [9]:
def demojizar(x):
    x = emoji.demojize(x, language='pt')
    return x

In [10]:
# Aplicando limpezas
tweets = tweets.str.lower()
tweets = tweets.apply(remover_url)
tweets = tweets.apply(remover_twiters)
tweets = tweets.apply(remover_pontuacao)
tweets = tweets.apply(demojizar)

In [11]:
# Função para remoção de stopwords e digitos
def limpar_texto(texto):
    doc = nlp(texto)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_digit]
    texto_limpo = " ".join(tokens)
    return texto_limpo

In [12]:
tweets = tweets.apply(limpar_texto)

# Tentativa de corrigir typos

Com essa biblioteca seria possivel corrigir erros de digitação.<br>
Em alguns tweets percebi que tinham palavras com letras repetidas ou contrações e isso poderia dificutar a nossa análise.<br>
Mas não consegui aplicar essa etapa da limpeza. Vou deixar marcado caso alguem queira tentar aplicar.

In [13]:
# Instanciando spellchecker
spell = SpellChecker(language='pt')

In [14]:
# Função para correção de grafia
def corretor_grafia(texto):
    tb = TextBlob(texto)
    corrected_words = []
    for word in tb.words:
        corrected_word = word.correct()
        corrected_words.append(corrected_word)
    
    return corrected_words

In [25]:
tweets

0        longo dia recebendo denúncias pessoas presas h...
1          simplesmente : coração_vermelho::coração_ver...
2          hahahhaha : coração_roxo::coração_roxo::cora...
3        longo dia recebendo denúncias pessoas presas h...
4                                                         
                               ...                        
22500          casa vc   opções mãe tias candidatas saí...
22501                                             sairão ♡
22502    candidatos pra estadual federal respectivamente  
22503                         força fabio : coração_roxo :
22504              ownnn conte comigo : coração_vermelho :
Name: rawContent, Length: 22505, dtype: object

In [15]:
tweets = tweets.apply(corretor_grafia)

KeyboardInterrupt: 

In [16]:
# Transformando em dataframe
df_tweets = tweets.to_frame()

In [17]:
df_tweets

,rawContent
1,MENÇÃO simplesmente : coração_vermelho::coraçã...
4,MENÇÃO
6,MENÇÃO : mão_beliscando_pele_clara::coração_ve...
10,MENÇÃO
12,MENÇÃO simplesmente : coração_vermelho::coraçã...
...,...
22498,ovotoésecreto \n deputada estadual MENÇÃO \n ...
22500,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO casa vc MEN...
22501,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO sairão ♡
22502,candidatos pra estadual federal respectivament...


In [18]:
# Identificando e excluindo linhas vazias. 
# Aqui não foi possível excluir nem identificar com funções NA, pq a linhas estava vazia.
df_tweets = df_tweets[df_tweets['rawContent'] != " "]

In [19]:
df_tweets

,rawContent
1,MENÇÃO simplesmente : coração_vermelho::coraçã...
4,MENÇÃO
6,MENÇÃO : mão_beliscando_pele_clara::coração_ve...
10,MENÇÃO
12,MENÇÃO simplesmente : coração_vermelho::coraçã...
...,...
22498,ovotoésecreto \n deputada estadual MENÇÃO \n ...
22500,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO casa vc MEN...
22501,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO sairão ♡
22502,candidatos pra estadual federal respectivament...


In [20]:
# Função para lemmatização
def lemmatizar(texto):
    doc = nlp(texto)
    lemmatized_tokens = []
    for token in doc:
        lemmatized_tokens.append(token.lemma_)
    return " ".join(lemmatized_tokens)

In [21]:
df_tweets['tweets_lemma'] = df_tweets['rawContent'].apply(lemmatizar)

In [22]:
df_tweets

,rawContent,tweets_lemma
1,MENÇÃO simplesmente : coração_vermelho::coraçã...,MENÇÃO simplesmente : coração_vermelho::coraçã...
4,MENÇÃO,MENÇÃO
6,MENÇÃO : mão_beliscando_pele_clara::coração_ve...,MENÇÃO : mão_beliscando_pele_clara::coração_ve...
10,MENÇÃO,MENÇÃO
12,MENÇÃO simplesmente : coração_vermelho::coraçã...,MENÇÃO simplesmente : coração_vermelho::coraçã...
...,...,...
22498,ovotoésecreto \n deputada estadual MENÇÃO \n ...,ovotoésecreto \n deputada estadual MENÇÃO \n...
22500,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO casa vc MEN...,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO casar vc ME...
22501,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO sairão ♡,MENÇÃO MENÇÃO MENÇÃO MENÇÃO MENÇÃO sair ♡
22502,candidatos pra estadual federal respectivament...,candidato pra estadual federal respectivamente...


In [20]:
# Função para classificação de palavras
def pos_tag(texto):
    doc = nlp(texto)
    pos_tags = []
    for token in doc:
        pos_tags.append((token.text, token.pos_))
    return pos_tags

In [21]:
df_tweets['tweets_pos'] = df_tweets['rawContent'].apply(pos_tag)

In [22]:
df_tweets['tweets_pos']

0        [(longo, ADJ), (dia, NOUN), (recebendo, VERB),...
1        [(  , SPACE), (simplesmente, ADV), (❤, NOUN), ...
2        [(  , SPACE), (hahahhaha, NOUN), (💜, PUNCT), (...
3        [(longo, ADJ), (dia, NOUN), (recebendo, VERB),...
5                              [(  , SPACE), (🫶🏾🫶🏾, NOUN)]
                               ...                        
22500    [(      , SPACE), (casa, NOUN), (vc, PROPN), (...
22501         [(      , SPACE), (sairão, VERB), (♡, NOUN)]
22502    [(candidatos, NOUN), (pra, ADP), (estadual, NO...
22503    [(   , SPACE), (força, NOUN), (fabio, PROPN), ...
22504    [(   , SPACE), (ownnn, NOUN), (conte, VERB), (...
Name: tweets_pos, Length: 21357, dtype: object

# Pipeline Completo
Para facilitar e aplicar todas as transformações e limpezas foi proposto pipeline, porém ele ainda não funciona.
Caso queiram aplicar mandem bala. Em uma das conversas com ChatGPT uma hipotese levantada foi de que o pipeline não suporta emojis e caracteres especial. Então precisariamos de uma limpeza maior. Ai entra outro ponto, se tirarmos todos esses caracteres podemos processar com mais eficacia, porém perderiamos informações no meio.

In [ ]:
# tfidf_vectorizer = TfidfVectorizer()

# def pos_tagging(text):
#     doc = nlp(text)
#     pos_tags = []
#     for token in doc:
#         pos_tags.append(token.pos_)
#     return pos_tags

# def lemmatize_text(text):
#     doc = nlp(text)
#     lemmatized_tokens = []
#     for token in doc:
#         lemmatized_tokens.append(token.lemma_)
#     return " ".join(lemmatized_tokens)

# def create_tfidf_matrix(texts):
#     tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
#     return tfidf_matrix

In [ ]:
# pipeline = Pipeline([
#     ('pos_tagging', FunctionTransformer(pos_tagging)),
#     ('lemmatization', FunctionTransformer(lemmatize_text)),
#     ('tfidf', FunctionTransformer(create_tfidf_matrix)),
# ])

In [ ]:
# preprocessed_data = pipeline.fit_transform(df_tweets_filt['rawContent'])

In [ ]:
# df_tweets['pos_tagged'] = df_tweets['rawContent'].apply(pos_tagging)
# df_tweets['lemma'] = df_tweets['pos_tagged'].apply(lemmatize_text)
# df_tweets['tfidf_matrix'] = df_tweets['lemma'].apply(create_tfidf_matrix)

# Modelos não supervisionados

## Clustering-based Approach (CBA)

In [27]:
# Tokenizar e preprocessar
tfidf = TfidfVectorizer(strip_accents='ascii', lowercase=True,
                        ngram_range=(1,5))
X = tfidf.fit_transform(df_tweets['tweets_lemma'])

# Treinar cluster para agrupar tweets similares
k = 2  # numero de clusters
kmeans = KMeans(n_clusters=k)
labels = kmeans.fit_predict(X)

# Rotular (não entendi exatamente porque e como estipularam 0 para positivo)
df_tweets['classif_cba'] = ['positive' if label == 0 else 'negative' for label in labels]

d:\ciencia_de_dados\projetos\tera_projeto\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [28]:
df_tweets['classif_cba'].value_counts()

positive    17798
negative     1592
Name: classif_cba, dtype: int64

## Topic Modeling-based Approach (TMBA)

In [29]:
# Tokenizar e preprocessar
cv = CountVectorizer()
X = cv.fit_transform(df_tweets['tweets_lemma'])

# Treinar LDA para identificar topicos/temas
n_topics = 2  # numero de topicos
lda = LatentDirichletAllocation(n_components=n_topics)
lda.fit(X)

# Manualmente atribuir rotulos para cada topico
topic_sentiments = ['positive', 'negative']
topic_labels = [topic_sentiments[i] for i in lda.transform(X).argmax(axis=1)]

# Rotular tweets baseado em topicos
df_tweets['classif_tmba'] = topic_labels

In [30]:
df_tweets['classif_tmba'].value_counts()

positive    12039
negative     9318
Name: classif_tmba, dtype: int64

## Sentiment Intensity Analysis (SIA)

In [31]:
nltk.download('vader_lexicon') # download do sentiment lexicon

sia = SentimentIntensityAnalyzer() # criar sentiment analyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\55199\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [33]:
# Loop pelos tweets e retornar score de sentimento
sentiments = []

for tweet in df_tweets['tweets_lemma']:
    sentiment = sia.polarity_scores(tweet)
    sentiments.append(sentiment)

In [35]:
df_tweets['sentiment_sia'] = sentiments

# Modelos supervisionados

## Sentiment140

In [46]:
# Função para classificação usando sentiment140
def classif_sent140(tweet):
    analise = TextBlob(tweet)
    if analise.sentiment.polarity > 0:
        return 'positivo'
    elif analise.sentiment.polarity == 0:
        return 'neutro'
    else:
        return 'negativo'

In [47]:
df_tweets['classif_sent140'] = df_tweets['tweets_lemma'].apply(classif_sent140)

In [49]:
df_tweets['classif_sent140'].value_counts()

neutro      20191
positivo      927
negativo      239
Name: classif_sent140, dtype: int64

## BERTimbau

In [50]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Carregar modelo pré treinado BERTimbau
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)

d:\ciencia_de_dados\projetos\tera_projeto\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ciencia_de_dados\projetos\tera_projeto\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\55199\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer

In [54]:
#atribuir análise ao CPU
device = torch.device('cpu')

# Função para analise sentimento
def predict_sentiment(tweet):
    # Encode do tweet com tokenizer do BERT
    inputs = tokenizer.encode_plus(tweet, add_special_tokens=True, return_tensors="pt")
    # Input IDs and attention mask (não entendi exatamente essa parte)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    # Fazer previsão com BERTimbau
    outputs = model(input_ids, attention_mask=attention_mask)
    # Rotulos de classificação (positivo ou negativo)
    predicted_label = torch.argmax(outputs[0]).item()
    # Returnar rotulos previstos
    return "positivo" if predicted_label == 1 else "negativo"


In [55]:
# Lista vazia para armazenar classificação de cada tweet
predicted_sentiments = []

# Loop por cada tweet e previsão
for tweet in df_tweets['tweets_lemma']:
    predicted_sentiment = predict_sentiment(tweet)
    predicted_sentiments.append(predicted_sentiment)

In [ ]:
df_tweets["classif_bertimbau"] = predicted_sentiments

In [56]:
df_tweets["classif_bertimbau"].value_counts()

negative    20571
positive      786
Name: predicted_sentiment, dtype: int64

In [60]:
df_tweets.rename(columns={'predicted_sentiment':'classif_bertimbau'}, inplace=True)

Nessa linha abaixo eu quis criar uma coluna com o compound da classificação SIA, pois o resultado do modelo é um dicionário com os valores que o modelo calculou para cada sentimento(neutro, positivo e negativo, sendo o compound uma metrica relativa aos 3 sentimentos presentes no texto)

In [98]:
list = []

for result in df_tweets['sentiment_sia']:
    compound = result['compound']
    list.append(compound)

SyntaxError: invalid syntax (4004766246.py, line 1)

In [82]:
df_tweets['compound_sia'] = list

In [91]:
df_tweets.rename(columns={'sentiment_sia':'classif_sia'}, inplace=True)

In [93]:
# Slavando resultado
df_tweets.to_csv('data/df_tweets_classif.csv', index_label=False)

In [94]:
# Carregando dataframe
df_novo = pd.read_csv('data/df_tweets_classif.csv')

In [95]:
df_novo

,rawContent,tweets_lemma,tweets_pos,classif_cba,classif_tmba,classif_sia,classif_sent140,classif_bertimbau,compound_sia
0,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...,"[('longo', 'ADJ'), ('dia', 'NOUN'), ('recebend...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
1,simplesmente ❤ ️ ❤ ️ ❤ ️,simplesmente ❤ ️ ❤ ️ ❤ ️,"[(' ', 'SPACE'), ('simplesmente', 'ADV'), ('❤...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
2,hahahhaha 💜 💜 💜,hahahhaha 💜 💜 💜,"[(' ', 'SPACE'), ('hahahhaha', 'NOUN'), ('💜',...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
3,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...,"[('longo', 'ADJ'), ('dia', 'NOUN'), ('recebend...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
5,🫶🏾🫶🏾,🫶🏾🫶🏾,"[(' ', 'SPACE'), ('\U0001faf6🏾\U0001faf6🏾', '...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
...,...,...,...,...,...,...,...,...,...
22500,casa vc opções mãe tias candidatas saí...,casa vc opção mãe tia candidato sair...,"[(' ', 'SPACE'), ('casa', 'NOUN'), ('vc',...",positive,positive,"{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compou...",neutro,negative,0.5106
22501,sairão ♡,sair ♡,"[(' ', 'SPACE'), ('sairão', 'VERB'), ('♡'...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
22502,candidatos pra estadual federal respectivamente,candidato pra estadual federal respectivamente,"[('candidatos', 'NOUN'), ('pra', 'ADP'), ('est...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
22503,força fabio 💜,força Fabio 💜,"[(' ', 'SPACE'), ('força', 'NOUN'), ('fabio'...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
